# 어떤 광고가 반응이 더 좋은가?

## 1. Loading data

#### 1) 필요한 라이브러리들을 파이썬으로 호출

In [ ]:
#데이터 분석에 필요한 패키지들을 파이썬으로 호출
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

#### 2) 변수 정의

In [ ]:
imp = pd.read_csv("ab_test_imp.csv", engine="python")
goal = pd.read_csv("ab_test_goal.csv", engine="python")

#### 3) 각 변수 확인

In [ ]:
imp.info()
imp[:5]

In [ ]:
goal.info()
goal[:5]

## 2. Data Cleaning

#### 1) 각 데이터 결합하기

In [ ]:
#두 데이터 결합하기 
ImpGoal = imp.merge(goal, on=["transaction_id"], how="outer")
ImpGoal.info()
ImpGoal[:10]

#### 2)일자별 정렬을 위한 날짜 처리

In [ ]:
#ImpGoal 데이터프레임 내 "log_date_x"시리즈를 datetime 유형으로 변경
ImpGoal['log_date_x']=ImpGoal['log_date_x'].apply(lambda x: pd.to_datetime(str(x)))

#### 3)광고별 클릭률 지표 설정

In [ ]:
#"is.goal" 시리즈를 만들고 "user_id_y"값이 "true"이면 0, 그렇지 않으면 1으로 표시
ImpGoal['is_goal'] = np.where(pd.isnull(ImpGoal["user_id_y"]) == True, 0, 1)

In [ ]:
ImpGoal.info()
ImpGoal[:10]

## 3. Data Analysis

#### 1)광고별 클릭률 확인

In [ ]:
#광고A와 광고B의 각 클릭률 비교
ImpGoal["is_goal"].groupby(ImpGoal["test_case_x"]).mean()

#### 2) 각 광고별 클릭률 트렌드 확인

In [ ]:
#"log_date_x" 기준에 따라 각 광고별 클릭률을 정렬
data = ImpGoal.groupby(["log_date_x", "test_case_x"])["is_goal"].mean()
data[:20]

#### 3) 각 광고별 클릭률 평균차이 검정(T-test)

In [ ]:
#unstack()함수를 이용해 완전한 데이터정규화 수행
data = data.unstack("test_case_x")

In [ ]:
#생성된 데이터프레임 확인
data.info()
data.head()

In [ ]:
#A광고와 B광고에 대한 집단 만들기
da = data["A"]
db = data["B"]

In [ ]:
#t-Test 수행 및 결과 도출
result = stats.ttest_ind(da, db)
print("t검정 통계량: %.3f, pvalue=%.3f"%(result))

## 4. Data Visuallization

#### 1) 각 광고별 클릭률 트렌드 시각화

In [ ]:
#완성된 데이터를 이용해 시각화 수행
data.plot.line(figsize=(15,7), style="--", marker="o", markersize=10, linewidth=2)